In [1]:
from inference import generate_response

# generate_response( """Read the question and give an honest answer. Your answers should not include any unethical, racist, sexist, dangerous, or illegal content. If the question is wrong, or does not make sense, accept it instead of giving the wrong answer.
# Question:{question}
# Choices: {choice}
# Answer:""")

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "college_computer_science")

In [3]:
opt = 8
question, choice, answer = ds['test']['question'][opt],ds['test']['choices'][opt] , ds['test']['answer'][opt]

In [4]:
print(generate_response( f"""
Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: {question}
Choices: {choice}
Answer (A, B, C, or D):"""))


Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: Which of the following statements about floating-point arithmetic is NOT true?
Choices: ['It is inherently nonassociative because some numbers have no exact representation.', 'It is inherently nonassociative because there have to be upper and lower bounds on the size of numbers.', 'Associativity can be achieved with appropriate roundoff conventions.', 'Some rational numbers have no exact representation.']
Answer (A, B, C, or D): A) A\t B) C\nB) D) E) F\ntl) Non-Amazons\tle\s? Answer: true: It is impossible to achieve the exact behavior that is represented by a floating point arithmetic., but it can still be accomplished by using simpler arithmetic rules. Additionally, floating points can also


In [5]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model.eval()

# Load the dataset
ds = load_dataset("cais/mmlu", "college_computer_science")
opt = 8
question, choices, correct_answer = ds['test']['question'][opt], ds['test']['choices'][opt], ds['test']['answer'][opt]

# Format the prompt
prompt = f"""
Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: {question}
Choices: {choices}
Answer (A, B, C, or D):
"""

# Tokenize the input and the correct answer
inputs = tokenizer(prompt, return_tensors="pt")
target_text = f" {correct_answer}"  # Space before to match GPT-2 tokenization
labels = tokenizer(target_text, return_tensors="pt")["input_ids"]

# Shift the labels to align with logits (causal language modeling loss requires this)
labels = torch.cat([torch.full((1, inputs.input_ids.size(1)), -100), labels], dim=1)

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss

print(f"Loss: {loss.item():.4f}")


/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ValueError: Expected input batch_size (125) to match target batch_size (126).

In [6]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model.eval()

# Load the dataset
ds = load_dataset("cais/mmlu", "college_computer_science")
opt = 8
question, choices, correct_answer = ds['test']['question'][opt], ds['test']['choices'][opt], ds['test']['answer'][opt]

# Format the prompt
prompt = f"""
Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: {question}
Choices: {choices}
Answer (A, B, C, or D):
"""

# Tokenize the input and the correct answer
inputs = tokenizer(prompt, return_tensors="pt")
target_text = f"{correct_answer}"  # No space before, GPT-2 adds a space by default to match tokenization
labels = tokenizer(prompt + target_text, return_tensors="pt")["input_ids"]

# Ensure labels align with input_ids
labels[0, :inputs.input_ids.size(1)] = -100  # Mask out the prompt tokens

# Forward pass through the model
with torch.no_grad():
    outputs = model(input_ids=inputs.input_ids, labels=labels)
    loss = outputs.loss

print(f"Loss: {loss.item():.4f}")


ValueError: Expected input batch_size (125) to match target batch_size (126).

In [7]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model.eval()

# Load the dataset
ds = load_dataset("cais/mmlu", "college_computer_science")
opt = 8
question, choices, correct_answer = ds['test']['question'][opt], ds['test']['choices'][opt], ds['test']['answer'][opt]

# Format the prompt
prompt = f"""
Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: {question}
Choices: {choices}
Answer (A, B, C, or D):
"""

# Tokenize the input and the correct answer
inputs = tokenizer(prompt, return_tensors="pt")
target_text = f"{correct_answer}"  # No space before, GPT-2 adds a space by default to match tokenization

# Tokenize the full input including the target
full_input = tokenizer(prompt + target_text, return_tensors="pt")

# Ensure input_ids and labels have the same batch size
input_ids = inputs.input_ids
labels = full_input.input_ids

# Mask out the prompt tokens
labels[0, :input_ids.size(1)] = -100

# Forward pass through the model
with torch.no_grad():
    outputs = model(input_ids=input_ids, labels=labels)
    loss = outputs.loss

print(f"Loss: {loss.item():.4f}")

ValueError: Expected input batch_size (125) to match target batch_size (126).

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

def generate_response(prompt, temperature=0.01, max_length=200):
    tokenizer = AutoTokenizer.from_pretrained("Arjun-G-Ravi/chat-GPT2")
    model = AutoModelForCausalLM.from_pretrained("Arjun-G-Ravi/chat-GPT2")
        
    encoding = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
        add_special_tokens=True
    )
    
    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask
    
    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,
    )
    
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def main():
    # Load the dataset
    ds = load_dataset("cais/mmlu", "college_computer_science")
    
    # Select a specific example
    opt = 8
    question = ds['test']['question'][opt]
    choices = ds['test']['choices'][opt]
    correct_answer = ds['test']['answer'][opt]
    
    # Format the prompt
    prompt = f"""Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: {question}
Choices: {choices}
Answer (A, B, C, or D):"""
    
    # Generate response
    response = generate_response(prompt)
    
    print("Prompt:")
    print(prompt)
    print("\nGenerated Response:")
    print(response)
    print("\nCorrect Answer:", correct_answer)

if __name__ == "__main__":
    main()

Prompt:
Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: Which of the following statements about floating-point arithmetic is NOT true?
Choices: ['It is inherently nonassociative because some numbers have no exact representation.', 'It is inherently nonassociative because there have to be upper and lower bounds on the size of numbers.', 'Associativity can be achieved with appropriate roundoff conventions.', 'Some rational numbers have no exact representation.']
Answer (A, B, C, or D):

Generated Response:
Read the question carefully and choose the correct answer by selecting one of the following options: A, B, C, or D.
Question: Which of the following statements about floating-point arithmetic is NOT true?
Choices: ['It is inherently nonassociative because some numbers have no exact representation.', 'It is inherently nonassociative because there have to be upper and lower bounds on the size of numbers.', 'Ass

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import numpy as np

def calculate_option_losses(model, tokenizer, question, choices):
    # Prepare the base prompt
    base_prompt = f"Question: {question}\nChoices: {choices}\nAnswer (A, B, C, or D):"
    
    # Losses for each option
    option_losses = {}
    
    # Iterate through options (A, B, C, D)
    for idx, option in enumerate(['A', 'B', 'C', 'D']):
        # Create full prompt with specific option
        full_prompt = base_prompt + f" {option}"
        
        # Tokenize the prompt
        inputs = tokenizer(full_prompt, return_tensors="pt", add_special_tokens=True)
        
        # Compute loss
        with torch.no_grad():
            outputs = model(
                input_ids=inputs['input_ids'], 
                attention_mask=inputs['attention_mask'], 
                labels=inputs['input_ids']
            )
            loss = outputs.loss.item()
        
        option_losses[option] = loss
    
    return option_losses

def benchmark_mmlu_dataset(subset='college_computer_science', num_samples=20):
    # Load the dataset
    ds = load_dataset("cais/mmlu", subset)
    
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = AutoModelForCausalLM.from_pretrained("gpt2")

    mapping = {0:'A', 1:'B', 2:'C', 3:'D'}
    
    # Prepare results storage
    results = {
        'total_samples': 0,
        'correct_predictions': 0,
        'option_performance': {'A': [], 'B': [], 'C': [], 'D': []}
    }
    
    # Iterate through test samples
    for i in range(min(num_samples, len(ds['test']))):
        # Extract question details
        question = ds['test']['question'][i]
        choices = ds['test']['choices'][i]
        correct_answer = mapping[ds['test']['answer'][i]]
        
        # Calculate losses for each option
        option_losses = calculate_option_losses(model, tokenizer, question, choices)
        
        # Determine predicted answer (lowest loss)
        predicted_answer = min(option_losses, key=option_losses.get)
        
        # Store results
        results['total_samples'] += 1
        results['correct_predictions'] += (predicted_answer == correct_answer)
        
        # Store option losses
        for option, loss in option_losses.items():
            results['option_performance'][option].append(loss)
        
        # Print detailed results for this sample
        print(f"\nSample {i+1}:")
        print(f"Question: {question}")
        print("Option Losses:")
        for option, loss in option_losses.items():
            print(f"{option}: {loss:.4f} {'(Correct)' if option == correct_answer else ''}")
        print(f"Predicted: {predicted_answer}, Correct: {correct_answer}")
    
    # Calculate and print summary statistics
    print("\n--- BENCHMARK SUMMARY ---")
    print(f"Total Samples: {results['total_samples']}")
    print(f"Accuracy: {results['correct_predictions'] / results['total_samples'] * 100:.2f}%")
    
    # Calculate mean losses for each option
    print("\nMean Option Losses:")
    for option, losses in results['option_performance'].items():
        print(f"{option}: {np.mean(losses):.4f} (std: {np.std(losses):.4f})")
    
    return results

def main():
    # Run benchmarking on computer science subset
    benchmark_mmlu_dataset()

if __name__ == "__main__":
    main()

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Sample 1:
Question: The access matrix approach to protection has the difficulty that
Option Losses:
A: 4.1106 (Correct)
B: 4.1504 
C: 4.1424 
D: 4.1524 
Predicted: A, Correct: A

Sample 2:
Question: An integer c is a common divisor of two integers x and y if and only if c is a divisor of x and c is a divisor of y. Which of the following sets of integers could possibly be the set of all common divisors of two integers?
Option Losses:
A: 1.7496 
B: 1.7716 
C: 1.7619 (Correct)
D: 1.7726 
Predicted: A, Correct: C

Sample 3:
Question: In the NoNicks operating system, the time required by a single file-read operation has four nonoverlapping components:
disk seek time-25 msec
disk latency time-8 msec
disk transfer time- 1 msec per 1,000 bytes
operating system overhead-1 msec per 1,000 bytes + 10 msec
In version 1 of the system, the file read retrieved blocks of 1,000 bytes. In version 2, the file read (along with the underlying layout on disk) was modified to retrieve blocks of 4,000 bytes. 